### Desafio 2 MeLi - Extração de Itens Via API MeLi

Este script faz parte de etapa do processo seletivo para o MeLi.

Através deste script, deseja-se extraír pelo menos 50 items para cada termo de entrada, para que sejam tratados e armazenados. Em resumo, o script:
- Lê os termos de entrada
- Realiza busca através de API aberta do Mercado Livre
- Para cada resultado, seu ID é extraído
- Para cada ID, é realizada uma consulta no endpoint /itens na API
- Os atributos mais relevantes são extraídos
- Os dados são organizados em um DataFrame
- É realizado um tratamento simples de remoção de duplicatas
- Os dados tratados são armazenados em uma planilha .csv

##### 1 - Importação das Bibliotecas

In [1]:
import requests
import pandas as pd

#### 2 - Definição dos Parâmetros

In [2]:
# Estes itens serão lidos pelo script
SEARCH_LIST = ['Google Home','Apple TV', 'Amazon Fire TV', 'chromecast', 'Alexa']

# Esta URL contém o Endpoint de busca dos itens pesquisados
URL_SEARCH_BASE = 'https://api.mercadolibre.com/sites/MLA'

# Esta URL contém a descrição detalhada de cada item
URL_ITEM_BASE = 'https://api.mercadolibre.com/items/'

# Este é o limite de extração para cada item
LIMIT = 50

# Este é o nome do arquivo de saída csv gerado
OUTPUT_NAME = 'oferta-vidriera.csv'

#### 3 - Busca dos Produtos

Aqui é realizado a extração dos IDs dos itens encontrados

In [3]:
item_list = []
searched_list = []

search_list = SEARCH_LIST
for search_product in search_list:
    url = f'{URL_SEARCH_BASE}/search?q={search_product}&limit={str(LIMIT)}#json'
    response = requests.get(url)
    data = response.json()
    results = data['results']
    for result in results:
        item_id = result['id']
        item_list.append(item_id)
        searched_list.append(search_product)

item_list[:5]

['MLA918072516',
 'MLA1146279904',
 'MLA1376911913',
 'MLA1389173603',
 'MLA1384134643']

#### 4 - Detalhes dos Produtos

Extraímos assim os detalhes dos produtos encontrado a partir do seu ID. Dados como nome, data de submissão, quantidade e preço são interessantes para a construção de um relatório de oferta dos produtos.

Foi adicionado também no catálogo o termo buscado para cada item encontrado, junto aos itens extraídos pela API

In [4]:
product_list = []
for item, searched_item in zip(item_list, searched_list):
    url_item = URL_ITEM_BASE + item
    response_item = requests.get(url_item)
    data_item = response_item.json()
    product_dict = {
        'id': data_item['id'],
        'item_search': searched_item,
        'title': data_item['title'],
        'condition':data_item['condition'],
        'domain': data_item['domain_id'],
        'price': data_item['price'],
        'original_price': data_item['original_price'],
        'date_created': data_item['date_created'],
        'date_updated' : data_item['last_updated'],
        'quantity': data_item['initial_quantity'],
        'link': data_item['permalink'],
        'status': data_item['status']
        }
    product_list.append(product_dict)

product_list[0]

{'id': 'MLA918072516',
 'item_search': 'Google Home',
 'title': 'Google Nest Mini Nest Mini 2nd Gen Con Asistente Virtual Google Assistant Chalk 110v/220v',
 'condition': 'new',
 'domain': 'MLA-SMART_SPEAKERS',
 'price': 61740.5,
 'original_price': 64990,
 'date_created': '2021-04-26T19:22:18.000Z',
 'date_updated': '2024-04-26T15:05:18.000Z',
 'quantity': 1935,
 'link': 'https://articulo.mercadolibre.com.ar/MLA-918072516-google-nest-mini-nest-mini-2nd-gen-con-asistente-virtual-google-assistant-chalk-110v220v-_JM',
 'status': 'active'}

#### 5 - Tratamento dos Dados

Aqui é realizado o tratamento dos dados. Novos tratamentos podem ser realizados, como ordenações, remoção de nulos, etc.

In [5]:
df_raw = pd.DataFrame(product_list)
df = df_raw.drop_duplicates(subset='id')
df.head(3)

,id,item_search,title,condition,domain,price,original_price,date_created,date_updated,quantity,link,status
0,MLA918072516,Google Home,Google Nest Mini Nest Mini 2nd Gen Con Asisten...,new,MLA-SMART_SPEAKERS,61740.5,64990.0,2021-04-26T19:22:18.000Z,2024-04-26T15:05:18.000Z,1935,https://articulo.mercadolibre.com.ar/MLA-91807...,active
1,MLA1146279904,Google Home,Google Home Mini Con Asistente Virtual Google ...,new,MLA-SMART_SPEAKERS,67190.0,99990.0,2022-07-09T21:58:24.000Z,2024-04-26T03:17:56.287Z,1271,https://articulo.mercadolibre.com.ar/MLA-11462...,active
2,MLA1376911913,Google Home,Google Home Mini Con Asistente Virtual Google ...,new,MLA-SMART_SPEAKERS,128999.0,NaN,2023-07-12T23:49:14.000Z,2024-04-17T17:40:46.000Z,20,https://articulo.mercadolibre.com.ar/MLA-13769...,active


#### 6 - Exportação dos Dados

In [6]:

df.to_csv(OUTPUT_NAME, index=False)